# Polynomial Regression model 

Gradient Descent

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib notebook  
pd.options.display.max_rows = 10
pd.options.display.max_columns=100

In [2]:
train=pd.read_csv('../data/kc_house_train_data.csv')
test=pd.read_csv('../data/kc_house_test_data.csv')

In [3]:
train['bedrooms_squared']=np.power(train['bedrooms'],2)
train['bed_bath_rooms']=np.multiply(train['bedrooms'],train['bathrooms'])
train['log_sqft_living']=np.log(train.sqft_living)
train['lat_plus_long']=train.lat+train.long

In [4]:
test['bedrooms_squared']=np.power(test['bedrooms'],2)
test['bed_bath_rooms']=np.multiply(test['bedrooms'],test['bathrooms'])
test['log_sqft_living']=np.log(test.sqft_living)
test['lat_plus_long']=test.lat+test.long

## Exploring data

In [5]:
test

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,bedrooms_squared,bed_bath_rooms,log_sqft_living,lat_plus_long
0,114101516,20140528T000000,310000.0,3,1.00,1430,19901,1.5,0,0,4,7,1430,0,1927,0,98028,47.7558,-122.229,1780,12697,9,3.00,7.265430,-74.4732
1,9297300055,20150124T000000,650000.0,4,3.00,2950,5000,2.0,0,3,3,9,1980,970,1979,0,98126,47.5714,-122.375,2140,4000,16,12.00,7.989560,-74.8036
2,1202000200,20141103T000000,233000.0,3,2.00,1710,4697,1.5,0,0,5,6,1710,0,1941,0,98002,47.3048,-122.218,1030,4705,9,6.00,7.444249,-74.9132
3,8562750320,20141110T000000,580500.0,3,2.50,2320,3980,2.0,0,0,3,8,2320,0,2003,0,98027,47.5391,-122.070,2580,3980,9,7.50,7.749322,-74.5309
4,7589200193,20141110T000000,535000.0,3,1.00,1090,3000,1.5,0,0,4,8,1090,0,1929,0,98117,47.6889,-122.375,1570,5080,9,3.00,6.993933,-74.6861
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4224,8672200110,20150317T000000,1088000.0,5,3.75,4170,8142,2.0,0,2,3,10,4170,0,2006,0,98056,47.5354,-122.181,3030,7980,25,18.75,8.335671,-74.6456
4225,5087900040,20141017T000000,350000.0,4,2.75,2500,5995,2.0,0,0,3,8,2500,0,2008,0,98042,47.3749,-122.107,2530,5988,16,11.00,7.824046,-74.7321
4226,3448900210,20141014T000000,610685.0,4,2.50,2520,6023,2.0,0,0,3,9,2520,0,2014,0,98056,47.5137,-122.167,2520,6023,16,10.00,7.832014,-74.6533
4227,6600060120,20150223T000000,400000.0,4,2.50,2310,5813,2.0,0,0,3,8,2310,0,2014,0,98146,47.5107,-122.362,1830,7200,16,10.00,7.745003,-74.8513


In [6]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17384 entries, 0 to 17383
Data columns (total 25 columns):
id                  17384 non-null int64
date                17384 non-null object
price               17384 non-null float64
bedrooms            17384 non-null int64
bathrooms           17384 non-null float64
sqft_living         17384 non-null int64
sqft_lot            17384 non-null int64
floors              17384 non-null float64
waterfront          17384 non-null int64
view                17384 non-null int64
condition           17384 non-null int64
grade               17384 non-null int64
sqft_above          17384 non-null int64
sqft_basement       17384 non-null int64
yr_built            17384 non-null int64
yr_renovated        17384 non-null int64
zipcode             17384 non-null int64
lat                 17384 non-null float64
long                17384 non-null float64
sqft_living15       17384 non-null int64
sqft_lot15          17384 non-null int64
bedrooms_squared   

In [7]:
test.describe()

,id,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,bedrooms_squared,bed_bath_rooms,log_sqft_living,lat_plus_long
count,4.229000e+03,4.229000e+03,4229.000000,4229.000000,4229.000000,4.229000e+03,4229.000000,4229.000000,4229.000000,4229.000000,4229.000000,4229.000000,4229.000000,4229.000000,4229.000000,4229.000000,4229.000000,4229.000000,4229.000000,4229.000000,4229.000000,4229.000000,4229.000000,4229.000000
mean,4.604771e+09,5.430540e+05,3.376921,2.113561,2079.366280,1.516886e+04,1.494561,0.007094,0.227004,3.403878,7.664460,1790.635848,288.730433,1970.398439,89.722629,98077.951762,47.563092,-122.216426,1988.844171,12735.877749,12.446678,7.503902,7.550275,-74.653334
std,2.894017e+09,3.564212e+05,1.021434,0.763560,905.317454,4.126563e+04,0.542292,0.083936,0.759375,0.654686,1.198476,832.215530,435.016314,29.552143,413.736867,53.426623,0.137965,0.140497,680.837632,27829.200218,17.868973,4.227032,0.426554,0.182162
min,1.000102e+06,8.500000e+04,0.000000,0.000000,370.000000,6.000000e+02,1.000000,0.000000,0.000000,1.000000,4.000000,370.000000,0.000000,1900.000000,0.000000,98001.000000,47.155900,-122.514000,700.000000,660.000000,0.000000,0.000000,5.913503,-75.177200
25%,2.110900e+09,3.250000e+05,3.000000,1.750000,1430.000000,5.027000e+03,1.000000,0.000000,0.000000,3.000000,7.000000,1180.000000,0.000000,1951.000000,0.000000,98032.000000,47.476600,-122.330000,1490.000000,5105.000000,9.000000,4.500000,7.265430,-74.767200
50%,4.019301e+09,4.530000e+05,3.000000,2.250000,1920.000000,7.633000e+03,1.500000,0.000000,0.000000,3.000000,7.000000,1570.000000,0.000000,1974.000000,0.000000,98065.000000,47.573400,-122.239000,1840.000000,7611.000000,9.000000,7.000000,7.560080,-74.667100
75%,7.338220e+09,6.500000e+05,4.000000,2.500000,2550.000000,1.076000e+04,2.000000,0.000000,0.000000,4.000000,8.000000,2230.000000,560.000000,1996.000000,0.000000,98118.000000,47.679500,-122.125000,2370.000000,10159.000000,16.000000,10.000000,7.843849,-74.528100
max,9.895000e+09,6.885000e+06,33.000000,7.750000,9890.000000,1.024068e+06,3.500000,1.000000,4.000000,5.000000,13.000000,8860.000000,2610.000000,2015.000000,2015.000000,98199.000000,47.777600,-121.315000,5030.000000,858132.000000,1089.000000,57.750000,9.199279,-73.602000


## Linear Regression - Gradient Descent

In [8]:
def predict_outcome(features_matrix,weights): 
    # features nXm
    # weights 1Xn
    # intercept 1
    n,_=features_matrix.shape
    assert weights.shape==(1,n)
    predictions=np.matmul(weights,features_matrix)
    return predictions

In [9]:
def feature_derivative(errors, feature):
    # features nXm
    # errors 1Xm
    assert errors.shape[1]==feature.shape[1]
    return np.matmul(errors,feature.T)

In [10]:
def regression_gradient_descent(feature_matrix, output, initial_weights, step_size, tolerance):
    # features_matrix nXm
    # output 1Xm
    n,m=feature_matrix.shape
    assert output.shape==(1,m)
    assert initial_weights.shape==(1,n)
    
    weights=np.array(initial_weights)
    for i in range(1000000):
        pred=predict_outcome(feature_matrix,weights)
        errors=output-pred
        MSS=sum(errors[0]**2)
        
        w_der=feature_derivative(errors,feature_matrix)
        weights+=step_size*w_der
        
        gradient_magnitude=np.sqrt(np.matmul(w_der[0].T,w_der[0]))

        if gradient_magnitude<tolerance:
            print "Stop at step {}".format(i)
            break
    return weights

In [11]:
# this was taken from Machine Learning MOOC by Andrew ng.
def closedForm(X,y):
    # you should catch np.linalg.LinAlgError Exception
    # X: nXm
    # y: 1Xm
    n,m=X.shape
    assert y.shape==(1,m)
    beta=np.matmul(np.matmul(np.linalg.inv(np.matmul(X,X.T)),X),y.T)
    return beta

In [12]:
def getData(data,features_names,target_name):
    features_matrix, output = train[features_names],train[target_name]
    output=output.reshape(1,output.shape[0])
    
    extra_column=np.ones((features_matrix.shape[0],1))
    features_matrix=np.concatenate([extra_column,features_matrix],axis=1)
    features_matrix=features_matrix.T

    n,m=features_matrix.shape
    assert output.shape==(1,m)
    return features_matrix,output

In [13]:
def meansquarederror(predictions,output):
    vec=predictions-output
    return  np.dot(vec,vec.T)/len(vec)

## using one feature

In [14]:
simple_features = ['sqft_living']
target_name= 'price'

initial_weights = np.array([[-47000.,1.]])

simple_feature_matrix, output = getData(train,simple_features,target_name)
step_size = 7e-12
tolerance = 2.5e7

#### Compare output parameter values between closedForm solution and gradient descent

In [15]:
m1_weights = regression_gradient_descent(simple_feature_matrix, output,initial_weights, step_size,                                             
                                             tolerance)
print m1_weights

Stop at step 14
[[-46999.88714469    281.91205132]]


In [16]:
m1_weights_CF=closedForm(simple_feature_matrix,output)
print "Using close form, weights are {}".format(m1_weights_CF)

Using close form, weights are [[-47116.07907289]
 [   281.95883963]]


#### predictions using one feature

In [17]:
test_simple_feature_matrix, test_output = getData(test,simple_features,target_name)

In [18]:
predicted_output=predict_outcome(test_simple_feature_matrix,m1_weights)

In [19]:
mse1=meansquarederror(test_output,predicted_output)
print "MSE=",mse1

MSE= [[1.20191839e+15]]


## using two features

In [20]:
model_features = ['sqft_living', 'sqft_living15'] # sqft_living15 is the average squarefeet for the nearest 15 neighbors. 
target_name = 'price'
(feature_matrix, output)  = getData(train,model_features,target_name)
initial_weights = np.array([[-100000., 1., 1.]])
step_size = 4e-12
tolerance = 1e9

#### Compare output parameter values between closedForm solution and gradient descent

In [21]:
m2_weights = regression_gradient_descent(feature_matrix, output,initial_weights, step_size,                                             
                                             tolerance)
print m2_weights

Stop at step 492
[[-9.99999687e+04  2.44960782e+02  6.54009515e+01]]


In [22]:
m2_weights_CF=closedForm(feature_matrix,output)
print m2_weights_CF

[[-1.00262175e+05]
 [ 2.45188714e+02]
 [ 6.52715852e+01]]


#### predictions using two feature

In [23]:
test_feature_matrix, test_output = getData(test,model_features,target_name)

In [24]:
predicted_output=predict_outcome(test_feature_matrix,m2_weights)

In [25]:
mse1=meansquarederror(test_output,predicted_output)
print "MSE=",mse1

MSE= [[1.18697834e+15]]
